In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\End_to_End_MLProject_with-MLFLOW\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\End_to_End_MLProject_with-MLFLOW'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/saisuryachandraprasad/End_to_End_MLProject_with-MLFLOW.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "saisuryachandraprasad"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "2dcd3ae6674d1754dae248d34dd8cce9a5c5aa0c"

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [8]:
@dataclass(frozen= True)

class ModelEvaluationconfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [9]:
from src.MLProject_With_MLFLOW.constants import *
from src.MLProject_With_MLFLOW.utils.common import read_yaml,create_directory, save_json

In [10]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])


    def get_model_evaluation_config(self):
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directory([config.root_dir])

        model_eveluation_config = ModelEvaluationconfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.target_column,
            mlflow_uri = "https://dagshub.com/saisuryachandraprasad/End_to_End_MLProject_with-MLFLOW.mlflow"

        )

        return model_eveluation_config

In [11]:
import os
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from urllib.parse import urlparse
import pandas as pd
import joblib

In [12]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationconfig) -> None:
        self.config = config


    def evaluate_metric(self,actual, predict):
        rmse = np.sqrt(mean_squared_error(actual, predict))
        mae =  mean_absolute_error(actual,predict)
        r2 = r2_score(actual, predict)

        return rmse,mae,r2
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():


            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2 ) = self.evaluate_metric(test_y,predicted_qualities)

            scores = {"rmse":rmse, "mae":mae, "r2_score": r2}

            save_json(path=Path(self.config.metric_file_name),data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2_score", r2)


            if tracking_url_type_store != "file":


                mlflow.sklearn.log_model(model, "model", registered_model_name= "ElasticNetModel")

            else:
                mlflow.sklearn.log_model(model, "model")




In [14]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config= model_evaluation_config)
    model_evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2023-11-01 15:18:03,160 - INFO - common - yaml_file config\config.yaml loaded successfully]
[2023-11-01 15:18:03,167 - INFO - common - yaml_file schema.yaml loaded successfully]
[2023-11-01 15:18:03,175 - INFO - common - yaml_file params.yaml loaded successfully]
[2023-11-01 15:18:03,179 - INFO - common - created directory atartifacts]
[2023-11-01 15:18:03,183 - INFO - common - created directory atartifacts/model_evaluation]


[2023-11-01 15:18:04,785 - INFO - common - data is lodaed to json file]


Registered model 'ElasticNetModel' already exists. Creating a new version of this model...
2023/11/01 15:18:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticNetModel, version 2
Created version '2' of model 'ElasticNetModel'.
